In [66]:
!pip3 install PyGithub requests

5000


In [92]:
def read_repos(txt):
    with open(txt, 'r') as f:
        repos = f.readlines()
        for i in range(len(repos)):
            repos[i] = repos[i].rstrip()
    return repos

def get_repo_names(repos):
    user_repo = []
    for i, repo in enumerate(repos):
        spl = repo.split("/")
        name_repo = spl[len(spl) - 1]
        username = spl[len(spl) - 2]
        user_repo.append([username, name_repo])
    return user_repo

def grab_issues(user_repo, client, count=None, requests=None):
    issues = []
    start_req = rate_limit(client)
    r_issues = user_repo.get_issues(state="all")
    totalCount = r_issues.totalCount
    for i in r_issues:
        issues.append(i)
        if count != None:
            if len(issues) == count:
                break
        if requests != None:
            if start_req - rate_limit(client) >= requests:
                break
    return issues, totalCount

def grab_commits(user_repo, client, count=None, requests=None):
    commits = []
    start_req = rate_limit(client)
    r_commits = user_repo.get_commits()
    totalCount = r_commits.totalCount
    for i in r_commits:
        commits.append(i)
        if count != None:
            if len(commits) == count:
                break
        if requests != None:
            if start_req - rate_limit(client) >= requests:
                break
    return commits, totalCount

def grab_wf(user_repo, client, count=None, requests=None):
    wf = []
    start_req = rate_limit(client)
    r_wf = user_repo.get_workflows()
    totalCount = r_wf.totalCount
    for i in r_wf:
        wf.append(i)
        if count != None:
            if len(wf) == count:
                break
        if requests != None:
            if start_req - rate_limit(client) >= requests:
                break
    return wf, totalCount

def grab_pr(user_repo, client, count=None, requests=None):
    pr = []
    start_req = rate_limit(client)
    r_pr = user_repo.get_pulls(state="all")
    totalCount = r_pr.totalCount
    for i in r_pr:
        pr.append(i)
        if count != None:
            if len(pr) == count:
                break
        if requests != None:
            if start_req - rate_limit(client) >= requests:
                break
    return pr, totalCount

def avg_time2reaction_and_mess_bug_reports(issues):
    bug_times = []
    mess_bug = 0
    mess_bug_num = 0
    for i in issues:
        if 'bug' in i.title:
            mess_bug +=i.comments
            mess_bug_num += 1
            if i.closed_at is not None:
                bug_times.append((i.closed_at - i.created_at).total_seconds())
    avg_bug_time = 0
    if len(bug_times) != 0:
        avg_bug_time = sum(bug_times)/len(bug_times)
    avg_mess_bug = mess_bug/mess_bug_num
    return avg_bug_time, avg_mess_bug

def avg_dur_pr(pullrequest):
    pr_times = []
    for pr in pullrequest:
        if pr.closed_at is not None:
            pr_times.append((pr.closed_at - pr.created_at).total_seconds())
    avg_pr_time = sum(pr_times)/len(pr_times)
    return avg_pr_time

def duration_of_pipelines(workflows, client, count=None, requests=None):
    w_time = []
    total_count = []
    start_req = rate_limit(client)
    for w in workflows:
        run_time = []
        runs = w.get_runs()
        total_count.append(runs.totalCount)
        for r in runs:
            try:
                t = r.timing().run_duration_ms
                run_time.append(t)
                if count != None:
                    if len(run_time) == count:
                        break
                if requests != None:
                    if start_req - rate_limit(client) >= requests/len(workflows):
                        break
            except Exception as e:
                print(e)
        w_time.append(run_time)
    avg_wt = []
    for wt in w_time:
        if len(wt) != 0:
            avg_wt.append(sum(wt)/len(wt)/1000)
    return avg_wt, total_count

def num_branches(user_repo):
    return user_repo.get_branches().totalCount

def commits_perday_perdev(commits):
    df = pd.DataFrame(columns=("author", "date"))
    for i, c in enumerate(commits):
        try:
            df.loc[i] = [c.commit.author.name] + [time.mktime(c.commit.author.date.timetuple())]
        except Exception as e:
            print(e)
    authors = set(df['author'])
    avg_mess_per_day_per_dev = []
    for author in authors:
        dates = df.iloc[df.index[df['author'] == author]]['date']
        delta = (dates.max() - dates.min())/86400
        if delta != 0:
            avg_mess_per_day_per_dev.append(len(dates)/delta)
    return avg_mess_per_day_per_dev

def contr_per_pr(pullrequest, client, count=None, requests=None):
    pull_users = []
    start_req = rate_limit(client)
    for i, pul in enumerate(pullrequest):
        comments = pul.get_comments()
        commits = pul.get_commits()
        users = []
        for comm in comments:
            try:
                if comm.user.name != None:
                    users.append(comm.user.name)
            except Exception as e:
                print(e)
            if requests != None:
                if start_req - rate_limit(client) >= requests:
                    break
        for comm in commits:
            if comm.commit.author.name != None:
                users.append(comm.commit.author.name)
            if requests != None:
                if start_req - rate_limit(client) >= requests:
                    break
        if count != None:
            if i == count:
                break
        if requests != None:
            if start_req - rate_limit(client) >= requests:
                break
        pull_users.append(users)
    contr_pull = []
    for u in pull_users:
        contr = []
        unique = set(u)
        for un in unique:
            contr.append(len(np.where(np.array(u) == un)[0]))
        contr_pull.append(contr)
    return contr_pull
        


from github import Github
import datetime
import time
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
# Github username

def rate_limit(client):
    return client.get_rate_limit().core.remaining

def generate_metrics(i):
    t = read_repos('tokens.txt')
    token = t[i%len(t)]
    client = Github(token)
    r_ = read_repos('repos.txt')
    user_repos = get_repo_names(r_)
    user = client.get_user(user_repos[i][0])
    user_repo = user.get_repo(user_repos[i][1])
    print(user_repos[i][0], user_repos[i][1])
    print("Rate limit at start:", rate_limit(client))
    print("Get Issues")
    issues, count_issues = grab_issues(user_repo, client, requests=50)
    print("Get Workflows")
    wf, count_wf = grab_wf(user_repo, client, requests=50)
    print("Get Pulls")
    pullreq, count_pullreq = grab_pr(user_repo, client, requests=250)
    print("Get Commits")
    commits, count_commits = grab_commits(user_repo, client, requests=150)
    print("Rate limit after grub:", rate_limit(client))
    print("Get Branches")
    num_branch = num_branches(user_repo)
    print("Get bug report lifetime, avg comments in bur report")
    avg_bug_time, avg_mess_bug = avg_time2reaction_and_mess_bug_reports(issues)
    print("Get Pull Request lifetime")
    avg_pr_time = avg_dur_pr(pullreq)
    print("Get bug comments per day per dev")
    avg_mess_per_day_per_dev = commits_perday_perdev(commits)
    r_lim = int(rate_limit(client)/6)
    print("Get CI/DI")
    avg_wt, count_avg_wt = duration_of_pipelines(wf, client, requests=600)
    print("Get Amount of contribution in Pull request")
    contr_pull = contr_per_pr(pullreq, client, requests=1200)
    metric_dict = {"user_repo":user_repos[i][0] + '_' + user_repos[i][1],
                   "count_issues":count_issues,
                   "count_wf":count_wf,
                   "count_pullreq":count_pullreq,
                   "count_commits":count_commits,
                   "num_branch":num_branch,
                   "avg_bug_time":avg_bug_time,
                   "avg_mess_bug":avg_mess_bug,
                   "avg_pr_time":avg_pr_time,
                   "avg_mess_per_day_per_dev":avg_mess_per_day_per_dev,
                   "avg_wt":avg_wt,
                   "count_avg_wt":count_avg_wt,
                   "contr_pull":contr_pull
                   }
    return metric_dict

In [93]:
res = generate_metrics(30)

spring-cloud spring-cloud-gateway
Rate limit at start: 4998
Get Issues
Get Workflows
Get Pulls
Get Commits
Rate limit after grub: 4860
Get Branches
Get bug report lifetime, avg comments in bur report
Get Pull Request lifetime
Get bug comments per day per dev
Get CI/DI
Get Amount of contribution in Pull request


In [94]:
git_metr_dict = {"Average time-to-reaction in bug reports (sec)":res['avg_bug_time'],
                 "Duration of CI/CD pipeline (sec)":sum([res['count_avg_wt'][i]*res['avg_wt'][i] for i in range(len(res['avg_wt']))]),
                 "Number of branches in Git":res['num_branch'],
                 "Average number of messages in a bug report":res['avg_mess_bug'],
                 "Average lifetime duration of a pull request (sec)":res['avg_pr_time'],
                 "Average Number of commits per day per developer":np.array([i for i in res['avg_mess_per_day_per_dev'] if i < np.percentile(res['avg_mess_per_day_per_dev'], 90)]).mean(),
                 "Number of commits":res['count_commits'],
                 "Average amount of contribution per pull request":np.sum(np.sum(res['contr_pull']))/len(res['contr_pull']),
                 "Amount of contribution of all pull requests":np.sum(np.sum(res['contr_pull']))/len(res['contr_pull'])*res['count_pullreq'],
                 "Count of pull requests":res['count_pullreq']
                }

In [95]:
git_metr_df = pd.DataFrame(git_metr_dict, index=[0])
git_metr_df.to_csv('csv/' + res['user_repo'].split('_')[1] + '/data/git.csv')

In [ ]:
results = Parallel(n_jobs=20)(delayed(generate_metrics)(i) for i in range(20))

GithubException: 500 null